In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ollama
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_community.llms import Ollama
import textwrap

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

def printText(text):
    wrapped_text = textwrap.fill(text, width=100) #text is the object which you want to print
    print(wrapped_text)

llm = Ollama(
    base_url='http://localhost:11434',
    model="llama3"
)

In [ ]:
from langchain.embeddings import OllamaEmbeddings
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = oembed
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
response = index.query(query, llm = llm)

In [ ]:
display(Markdown(response))

## Step by Step

In [ ]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()
docs[0]

In [ ]:
embed = oembed.embed_query("Hi my name is Harrison")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    oembed
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
retriever = db.as_retriever()

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [ ]:
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."
response = qa_stuff.run(query)
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)
response

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=oembed,
).from_loaders([loader])